# A Network Simplex Energy Systems Model

What if we could re-formulate MILP energy systems models as a minimum-cost-flow problem? This would allow us to use super-efficient simplex methods to solve for energy flow, basically creating the minimum-cost sankey diagram through time.

**Advantages:**
- model using plant-based data
- flexible region-sector-time definition
- min-cost simplex problem

**Trade-offs:**
- integer-value capacity-expansions

**Questions:**
- emissions and other constraints?
- operating models?
- storage?

![alternative text](netflow_esys_model.png)

### Let's demonstrate the capacity expansion mechanism first

**Problem Formulation:**
Let's find the minimum cost flow for a graph, etc. etc.

![alternative text](simple_toy.png)

In [2]:
from pydantic import BaseModel
from itertools import chain, product

**Define some plants / technologies**

In [3]:
from typing import Optional

In [4]:
class Plant(BaseModel):
    id:str                       # id
    remaining_life: int          # years
    output_capacity: int                # kW
    io_ratio: float              # efficiency
    capex: int                   # $
    opex: int                    # $/KWh
    commodity_input: str         # INPUT commodity
    commodity_output: str        # OUTPUT commodity
    availability_year: int       # first year
    inp_nodes: Optional[list] = None
    outp_nodes: Optional[list] = None
    capexp_node: Optional[str] = None
    active_yrtimes: Optional[list] = None
    input_capacity: int = None

In [5]:
initial_plants = [
    Plant(
        id='plant-existing-0-0',
        remaining_life=5,
        output_capacity = 2000,
        io_ratio = 0.8,
        capex = 0,
        opex = 1,
        commodity_input = "INPUT",
        commodity_output = "OUTPUT",
        availability_year = 0,
        input_capacity = int(2000/0.8),
    ),
    Plant(
        id='plant-existing-0-1',
        remaining_life=15,
        output_capacity = 5000,
        io_ratio = 0.6,
        capex = 0,
        opex=2,
        commodity_input = "INPUT",
        commodity_output = "OUTPUT",
        availability_year = 0,
        input_capacity = int(5000/0.6),
    )
]

In [6]:
new_plants = [
    [
        Plant(
            id=f'plant-new-{year}-0',
            remaining_life=10,
            output_capacity=1000,
            io_ratio = 0.8-0.2*(year/30),
            capex = 1500*1000,
            opex = 1,
            availability_year = year,
            commodity_input = "INPUT",
            commodity_output = "OUTPUT",
            input_capacity = int(1000/( 0.8-0.2*(year/30))),
        ),
        Plant(
            id=f'plant-new-{year}-1',
            remaining_life=20,
            output_capacity=2000,
            io_ratio =  0.6-0.1*(year/30),
            capex = 1000*2000,
            opex = 2,
            availability_year = year,
            commodity_input = "INPUT",
            commodity_output = "OUTPUT",
            input_capacity = int(2000/(0.6-0.1*(year/30))),
        ),
    ]
    for year in range(30)
]
all_plants = initial_plants + list(chain(*new_plants))

**Set some demands**

In [7]:
times = range(12)
years = range(30)
peak_demands = {yr:6000+500*yr for yr in years}
yr_time_demand = {f'{yr}-{t}':int(0.5*peak_demands[yr]+0.5*peak_demands[yr]*(1-(abs((t-6)/6)))) for yr, t in product(years, times)}
commodities = ["INPUT","OUTPUT"]

### BUILD THE GRAPH

In [8]:
import networkx as nx

In [9]:
G = nx.DiGraph()

**Add all the Nodes**

In [10]:
# commodity buses
commodity_nodes = ['-'.join([commodity,yr_time]) for commodity, yr_time in product(commodities, yr_time_demand.keys())]

In [11]:
G.add_nodes_from(commodity_nodes)

In [12]:
# add existing plants and new plants
for plant in all_plants:
    # add input nodes
    plant.active_yrtimes = [[yr,t] for yr, t in product(range(plant.availability_year, min(30,plant.availability_year+plant.remaining_life)), times)]
    #inp_nodes = [f'{plant.id}-{yr}-{t}-{plant.commodity_input}' for yr, t in product(range(plant.availability_year, min(30,plant.availability_year+plant.remaining_life)), times)]
    # add output nodes
    #outp_nodes = [f'{plant.id}-{yr}-{t}-{plant.commodity_output}' for yr, t in product(range(plant.availability_year, min(30,plant.availability_year+plant.remaining_life)), times)]
    # add cap_exp_node
    # add_nodes = inp_nodes + outp_nodes + [f'{plant.id}-capexp']
    # plant.inp_nodes = inp_nodes
    # plant.outp_nodes = outp_nodes
    # plant.capexp_node = f'{plant.id}-capexp'
    plant_nodes = [f'{plant.id}-{yr}-{t}' for yr, t in product(range(plant.availability_year, min(30,plant.availability_year+plant.remaining_life)), times)]
    G.add_nodes_from(plant_nodes)

In [13]:
plants_dict = {p.id: p for p in all_plants}

In [14]:
# add optimisation nodes
global_demand = sum(yr_time_demand.values())
LARGE_VAL = global_demand*100
optim_nodes = [
    ('GLOBAL_DEMAND',{'demand':global_demand}),
    ('GLOBAL_SUPPLY',{'demand':-LARGE_VAL}),
    ('LOSS_SINK',{'demand':LARGE_VAL-global_demand}),
    # ('CAPEXP_SOURCE',{'demand':-LARGE_VAL}),
    # ('CAPEXP_SINK',{'demand':LARGE_VAL}),
]
G.add_nodes_from(optim_nodes)

**Add all the edges**

*... here be dragons...*

In [15]:
### ENERGY FLOW
# input & output edges
input_edges = [ ('GLOBAL_SUPPLY','-'.join(['INPUT',yr_time]),{'capacity':LARGE_VAL,'cost_var':0,'cost_fixed':0}) for yr_time in yr_time_demand.keys()]
output_edges = [('-'.join(['OUTPUT',yr_time]),'GLOBAL_DEMAND',{'capacity':val, 'cost_var':0, 'cost_fixed':0}) for yr_time, val in yr_time_demand.items()]
G.add_edges_from(input_edges+output_edges)

In [16]:
# plant endges
for plant in all_plants:
    input_edges =  [('-'.join([plant.commodity_input,str(yr),str(t)]), f'{plant.id}-{yr}-{t}', {'capacity':plant.input_capacity, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]# inp-commod to inp-plant
    # bridge_edges = [(f'{plant.id}-{yr}-{t}-{plant.commodity_input}',f'{plant.id}-{yr}-{t}-{plant.commodity_output}', {'capacity':plant.output_capacity-1, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    output_edges = [(f'{plant.id}-{yr}-{t}','-'.join([plant.commodity_output,str(yr),str(t)]), {'capacity':plant.output_capacity, 'cost_var':plant.opex, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    loss_edges = [(f'{plant.id}-{yr}-{t}','LOSS_SINK', {'capacity':plant.input_capacity-plant.output_capacity, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    # capexp_minus_edges = [(f'{plant.id}-{yr}-{t}-{plant.commodity_input}','CAPEX_SINK',{'capacity':1, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    # capexp_plus_edges = [(plant.capexp_node,f'{plant.id}-{yr}-{t}-{plant.commodity_input}',{'capacity':1, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    # capexp_edge = [('CAPEXP_SOURCE',plant.capexp_node,{'capacity':LARGE_VAL, 'cost_var':0, 'cost_fixed':plant.capex})]
    G.add_edges_from(input_edges+output_edges+loss_edges)

In [17]:
# add bypasses
#supply_bypass = ('GLOBAL_SUPPLY','LOSS_SINK',{'capacity':LARGE_VAL,'cost_var':LARGE_VAL*1e9,'cost_fixed':0})
#capexp_bypass = ('CAPEXP_SOURCE','CAPEXP_SINK',{'capacity':LARGE_VAL,'cost_var':0,'cost_fixed':0})
#G.add_edges_from([supply_bypass, capexp_bypass])

In [18]:
from pulp import *

In [19]:
model = LpProblem("graph_model_mincostflow",LpMinimize)

In [20]:
# variables: only 2; 
F = LpVariable.dicts("Flows",list(G.edges), lowBound=0, cat='continuous')
capexp = LpVariable.dicts("CapExp", [p.id for p in all_plants], 0,1, LpInteger)

In [21]:
### CONSTRAINTS ###

# for every node, sum(flows) + demand == 0
for n, params in G.nodes.items():
    model += sum([F[e_i] for e_i in G.in_edges(n)])-sum([F[e_o] for e_o in G.out_edges(n)]) - params.get('demand',0) == 0

In [22]:
# for every edge, constrain capacity
for (n_s, n_t), data in G.edges.items():
    if 'plant' in n_s:
        p_id = '-'.join(n_s.split('-')[0:4])
        model += F[(n_s, n_t)] <= data['capacity'] * capexp[p_id]
    elif 'plant' in n_t:
        p_id = '-'.join(n_t.split('-')[0:4])
        model += F[(n_s, n_t)] <= data['capacity'] * capexp[p_id]
    else:
        model += F[(n_s, n_t)] <= data['capacity']

In [23]:
# for each plant node, the LOSS_SINK flow == sum(flow_inp) * io_ratio
for n_s, data in G.nodes.items():
    if 'plant' in n_s:
        p_id = '-'.join(n_s.split('-')[0:4])
        model += F[(n_s, 'LOSS_SINK')] == sum([F[e_i] for e_i in G.in_edges(n_s)]) * (1. - plants_dict[p_id].io_ratio)

In [26]:
# sum costs
flow_costs = [F[(n_s, n_t)] * e_data['cost_var'] for (n_s, n_t), e_data in G.edges.items()]
capexp_costs = [capexp[p_id] * plants_dict[p_id].capex for p_id in capexp.keys()]
total_cost = sum(flow_costs) + sum(capexp_costs)
model += total_cost, "Total Cost"

In [25]:
GUROBI_CMD()

In [ ]:
model.solve(GUROBI_CMD())

In [ ]:
print(LpStatus[model.status])

In [ ]:
for kk,vv in capexp.items():
    print (kk,vv.value())

In [ ]:
for kk,vv in F.items():
    print (kk,vv.value())

In [ ]:
### MILP
# for every node, sum(flows)-demand==0
# for every edge, flow<=capacity * capexp
# for loss edge, flow==flow_inp*io_ratio
# costs = sum(flow_costs)+capexp